# Toronto Neighborhoods Analysis

## Description

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


## Collecting Data

### Data Source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Target: Data Frame
* 3 columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. (Borough is not null)
* More than one neighborhood can exist in one postal code area. 
    * For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
    * So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Get data from website

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Tue, 16 Apr 2019 16:51:03 GMT', 'Server-Timing': 'cache;desc="hit-front"', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Backend-Timing': 'D=106893 t=1555265421068791', 'Set-Cookie': 'WMF-Last-Access=16-Apr-2019;Path=/;HttpOnly;secure;Expires=Sat, 18 May 2019 12:00:00 GMT, WMF-Last-Access-Global=16-Apr-2019;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Sat, 18 May 2019 12:00:00 GMT, GeoIP=US:::37.75:-97.82:v4; Path=/; secure; Domain=.wikipedia.org', 'Content-language': 'en', 'X-Content-Type-Options': 'nosniff', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate', 'Content-Type': 'text/html; charset=UTF-8', 'Connection': 'keep-alive', 'X-Cache': 'cp1081 hit/4, cp2012 hit/5, cp2023 hit/2', 'X-Analytics': 'ns=0;page_id=539066;https=1;nocookies=1', 'Last-Modified': 'Tue, 02 Apr 2019 21:01:21 GMT', 'Server': 'mw12

## Parse data from online

#### Soup

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    row = tr.find_all('td')
    if row:
        rows.append(row)
# rows

In [4]:
lst = []
for row in rows:
    postalcode = row[0].text.strip()
    borough = row[1].text.strip()
    neighborhood = row[2].text.strip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
# lst

#### Convert to pandas

In [5]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns = cols)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### Group df by PostalCode

In [6]:
df = df.groupby('PostalCode').agg(
    {
          'Borough': 'first'
        , 'Neighborhood': ', '.join
    }
).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


### DF shape

In [8]:
df.shape

(103, 3)